In [8]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('Basel_8')
# print(latex_text(cl))

ModuleNotFoundError: No module named 'Circle'

NameError: name 'ClusterDias' is not defined

In [47]:
%run runaway_functionsv3

cl = Cluster('Basel_8')
cld = ClusterDias('Basel_8')
obs = config['observed_stars']['Basel_8']

plot_cmd = cl.plot_cmd(isochrones=[Isochrone(cld)])
show_annotation_manager(plot_cmd)


   Name  
---------
HD 259597
   Name  
---------
HD 259787
   Name  
---------
HD 259991


In [10]:
plot_cmd = cl.plot_cmd()
show_annotation_manager(plot_cmd)

In [48]:
plot_pm = cl.plot_pm()
show_annotation_manager(plot_pm)

In [5]:
%run runaway_functionsv3
%matplotlib qt

cl = Cluster('Basel_8')
plot_traceback_clean = cl.plot_traceback_clean()
show_annotation_manager(plot_traceback_clean)

In [7]:
scatter = plot_traceback_clean.scatter(5,512,s=500, c='white')


AttributeError: 'PathCollection' object has no attribute 'Circle'

In [7]:
%run runaway_functionsv3
%matplotlib qt
cl  = Cluster('Basel_8')
plot_traceback_psr = cl.plot_traceback_psr()


(array([154.76217518,  76.12140822, 238.72218691, 178.64062384,
       218.35386502]), array([ 85.63831627, 136.63417909, 245.01511754, 259.75616726,
       109.94681138]))


In [5]:
circle_v_spr = patches.Circle((500,500),radius=50,edgecolor='azure', facecolor='aqua',alpha = 0.1)
plot_traceback_psr.add_patch(circle_v_spr)

In [6]:
cl = Cluster('Alessi_43')
cl.runaways()

Name,RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,e_v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,K,K,km / s,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
Cl* Trumpler 10 STOCK 774,132.26918571419,-41.86012579175,814.32153300,8231.5,10290.0,23.30687698503049,1.6444227372214824,26.05,8.25,--,--,5524602036116091520,1.1904,0.0254,-10.707,7.029,0.026,0.029,1.723,11.258210,0.356719,11.384149,11.027430,796.94012500,831.44531200,0.002774,0.002865,0.003795,0.00666,"132.26918571419,-41.86012579175,814.321533",-5.176,0.297,3.109,0.300,4.0056,31.10,2.85,11.63,12.13,591102.51261,175.7,3.817e+05,229,3.077e+05,96.41,--,--


In [23]:
(cl.Star(3326048448124872320)[0])

Name,RA_ICRS_1,DE_ICRS_1,rgeo,b_rgeo,B_rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,pc,pc,K,K,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str10,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
HD 259597,98.38429327278,8.33557580226,1517.70947000,1442.38904000,1621.94727000,32036.6,13110.0,9.081153222849567,2.36655530497063,--,--,733-1218-1,3326048448124872320,0.6283,0.0363,-0.760,-1.057,0.035,0.028,1.363,8.432107,0.337074,8.528219,8.191145,0.00381,0.009581,0.01014,0.01972,"98.38429327278,8.33557580226,1517.70947",-0.570,0.260,1.126,0.253,3.9984,--,--,--,--,7981856.09929,1.935e+04,5.298e+06,4.473e+04,4.195e+06,3.635e+04,--,--


In [45]:
%run runaway_functionsv3
cl = Cluster('Basel_8')
cld = ClusterDias('Basel_8')
obs = config['observed_stars']['Basel_8']

cl.plot_cmd(isochrones=[Isochrone(cld)])


getting isochrone from cmd3.7 (PARSEC2) with:
Av:1.08
logage:7.67
metallicity:0.03
isochrone downloaded in 7.1s
   Name  
---------
HD 259597
   Name  
---------
HD 259787
   Name  
---------
HD 259991


<AxesSubplot: xlabel='$G_{BP}-G_{RP}$ (mag)', ylabel='$G$ (mag)'>